In [ ]:
"""
Generate products.csv for Data Science Course
==============================================
Uses Faker library for realistic product data
Optimized for speed with vectorized operations

Run: pip install faker pandas numpy
"""

import pandas as pd
import numpy as np
from faker import Faker
import time

# Initialize
fake = Faker()
Faker.seed(42)
np.random.seed(42)

print("="*70)
print("GENERATING PRODUCTS.CSV")
print("="*70)

# Configuration
NUM_PRODUCTS = 1000

start_time = time.time()
print(f"\nGenerating {NUM_PRODUCTS:,} products...")

# ============================================================================
# PRODUCT CATEGORIES AND HIERARCHY
# ============================================================================

print("  → Setting up product categories...")

# Main categories with realistic distribution
categories_config = {
    'Electronics': {
        'weight': 0.15,
        'subcategories': ['Mobile Phones', 'Laptops', 'Tablets', 'Headphones', 'Cameras', 'Smartwatches'],
        'price_range': (5000, 150000),
        'avg_rating': 4.2
    },
    'Clothing': {
        'weight': 0.12,
        'subcategories': ['Men', 'Women', 'Kids', 'Accessories', 'Footwear'],
        'price_range': (300, 8000),
        'avg_rating': 4.0
    },
    'Home & Kitchen': {
        'weight': 0.10,
        'subcategories': ['Furniture', 'Appliances', 'Decor', 'Cookware', 'Bedding'],
        'price_range': (500, 80000),
        'avg_rating': 4.1
    },
    'Books': {
        'weight': 0.08,
        'subcategories': ['Fiction', 'Non-Fiction', 'Academic', 'Comics', 'Children'],
        'price_range': (150, 3000),
        'avg_rating': 4.3
    },
    'Sports & Fitness': {
        'weight': 0.08,
        'subcategories': ['Fitness Equipment', 'Outdoor', 'Team Sports', 'Yoga', 'Cycling'],
        'price_range': (300, 50000),
        'avg_rating': 4.0
    },
    'Beauty & Personal Care': {
        'weight': 0.10,
        'subcategories': ['Skincare', 'Makeup', 'Haircare', 'Fragrances', 'Bath & Body'],
        'price_range': (200, 5000),
        'avg_rating': 4.1
    },
    'Toys & Games': {
        'weight': 0.07,
        'subcategories': ['Educational', 'Electronic', 'Outdoor', 'Board Games', 'Action Figures'],
        'price_range': (150, 8000),
        'avg_rating': 4.2
    },
    'Automotive': {
        'weight': 0.08,
        'subcategories': ['Parts', 'Accessories', 'Tools', 'Care Products', 'Electronics'],
        'price_range': (300, 50000),
        'avg_rating': 3.9
    },
    'Grocery & Gourmet': {
        'weight': 0.12,
        'subcategories': ['Snacks', 'Beverages', 'Staples', 'Organic', 'International'],
        'price_range': (50, 2000),
        'avg_rating': 4.0
    },
    'Health & Wellness': {
        'weight': 0.10,
        'subcategories': ['Supplements', 'Medical Devices', 'Wellness', 'Personal Care', 'Ayurveda'],
        'price_range': (200, 8000),
        'avg_rating': 4.1
    }
}

# Calculate number of products per category
categories = list(categories_config.keys())
weights = [categories_config[cat]['weight'] for cat in categories]
products_per_category = (np.array(weights) * NUM_PRODUCTS).astype(int)

# Adjust to ensure exact NUM_PRODUCTS
diff = NUM_PRODUCTS - products_per_category.sum()
products_per_category[0] += diff

# ============================================================================
# GENERATE PRODUCT DATA
# ============================================================================

print("  → Generating product attributes...")

product_data = []
product_id = 1

for category, num_products in zip(categories, products_per_category):
    config = categories_config[category]
    
    for _ in range(num_products):
        # Subcategory
        subcategory = np.random.choice(config['subcategories'])
        
        # Product name (more realistic)
        brand = fake.company().split()[0]  # Take first word as brand
        product_type = subcategory.split()[0] if ' ' in subcategory else subcategory
        
        # Generate realistic product name
        if category == 'Electronics':
            model = f"{np.random.choice(['Pro', 'Max', 'Plus', 'Ultra', 'Lite'])}"
            name = f"{brand} {product_type} {model} {fake.color_name()}"
        elif category == 'Clothing':
            style = np.random.choice(['Casual', 'Formal', 'Sport', 'Premium', 'Classic'])
            name = f"{brand} {style} {product_type}"
        elif category == 'Books':
            name = fake.catch_phrase()  # Random book title
        else:
            name = f"{brand} {product_type} {fake.word().capitalize()}"
        
        # Price based on category range with log-normal distribution
        min_price, max_price = config['price_range']
        mean_price = (min_price + max_price) / 2
        price = np.random.lognormal(mean=np.log(mean_price), sigma=0.5)
        price = int(np.clip(price, min_price, max_price))
        
        # Rating (mostly good, some bad - realistic)
        # Using beta distribution centered around category average
        avg_rating = config['avg_rating']
        rating = np.random.beta(a=8, b=2) * 4 + 1  # Range 1-5, skewed positive
        rating = round(np.clip(rating, 1.0, 5.0), 1)
        
        # Number of reviews (correlated with rating)
        if rating >= 4.0:
            num_reviews = int(np.random.lognormal(mean=4, sigma=1.5))
        else:
            num_reviews = int(np.random.lognormal(mean=2.5, sigma=1.5))
        num_reviews = np.clip(num_reviews, 0, 10000)
        
        # Stock quantity (some out of stock)
        if np.random.random() < 0.05:  # 5% out of stock
            stock = 0
        else:
            stock = int(np.random.lognormal(mean=4, sigma=1.5))
            stock = np.clip(stock, 1, 1000)
        
        # Discount (seasonal/clearance)
        discount_prob = np.random.random()
        if discount_prob < 0.50:  # 50% no discount
            discount = 0
        elif discount_prob < 0.70:  # 20% small discount
            discount = np.random.choice([5, 10])
        elif discount_prob < 0.85:  # 15% medium discount
            discount = np.random.choice([15, 20])
        else:  # 15% large discount
            discount = np.random.choice([25, 30, 40, 50])
        
        # Weight (for logistics - in kg)
        if category in ['Electronics', 'Home & Kitchen', 'Sports & Fitness']:
            weight = round(np.random.lognormal(mean=1, sigma=1), 2)
            weight = np.clip(weight, 0.1, 50)
        else:
            weight = round(np.random.lognormal(mean=-1, sigma=1), 2)
            weight = np.clip(weight, 0.05, 10)
        
        # Dimensions (L x W x H in cm)
        base_dim = weight ** (1/3) * 10  # Rough correlation with weight
        length = round(base_dim * np.random.uniform(1.2, 2.0), 1)
        width = round(base_dim * np.random.uniform(0.8, 1.2), 1)
        height = round(base_dim * np.random.uniform(0.6, 1.5), 1)
        
        # Is featured product (top rated, in stock)
        is_featured = (rating >= 4.3 and stock > 10 and np.random.random() < 0.15)
        
        # Color (for applicable categories)
        if category in ['Electronics', 'Clothing', 'Home & Kitchen']:
            color = fake.color_name()
        else:
            color = 'N/A'
        
        # Warranty months
        if category in ['Electronics', 'Home & Kitchen', 'Automotive']:
            warranty = np.random.choice([6, 12, 24, 36], p=[0.2, 0.5, 0.2, 0.1])
        else:
            warranty = 0
        
        # Image path (for CNN tasks)
        image_path = f"images/products/{category.lower().replace(' ', '_')}/prod_{product_id:05d}.jpg"
        
        product_data.append({
            'Product_ID': f'PROD_{product_id:05d}',
            'Product_Name': name[:100],  # Limit length
            'Category': category,
            'Subcategory': subcategory,
            'Brand': brand,
            'Price': price,
            'Discount_Percent': discount,
            'Final_Price': int(price * (1 - discount/100)),
            'Rating': rating,
            'Number_of_Reviews': num_reviews,
            'Stock_Quantity': stock,
            'Weight_kg': weight,
            'Length_cm': length,
            'Width_cm': width,
            'Height_cm': height,
            'Color': color,
            'Warranty_Months': warranty,
            'Is_Featured': is_featured,
            'Image_Path': image_path
        })
        
        product_id += 1

# ============================================================================
# CREATE DATAFRAME
# ============================================================================

print("  → Assembling dataframe...")
products_df = pd.DataFrame(product_data)

# Add launch date (when product was added to catalog)
from datetime import datetime, timedelta
start_date = datetime(2019, 1, 1)
end_date = datetime(2024, 12, 31)
days_range = (end_date - start_date).days

launch_days = np.random.beta(a=2, b=5, size=NUM_PRODUCTS) * days_range
products_df['Launch_Date'] = [start_date + timedelta(days=int(d)) for d in launch_days]

# ============================================================================
# ADD MISSING VALUES (REALISTIC)
# ============================================================================

print("  → Adding realistic missing values...")

# Some products don't have dimensions/weight (digital products, etc.)
missing_dims = np.random.choice(NUM_PRODUCTS, size=int(NUM_PRODUCTS * 0.05), replace=False)
products_df.loc[missing_dims, ['Weight_kg', 'Length_cm', 'Width_cm', 'Height_cm']] = np.nan

# Some products missing color info
missing_color = np.random.choice(NUM_PRODUCTS, size=int(NUM_PRODUCTS * 0.10), replace=False)
products_df.loc[missing_color, 'Color'] = np.nan

# ============================================================================
# SAVE TO CSV
# ============================================================================

print("  → Saving to products.csv...")
products_df.to_csv(r'./products.csv', index=False)

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

elapsed_time = time.time() - start_time

print("\n" + "="*70)
print("✅ PRODUCTS.CSV GENERATED SUCCESSFULLY!")
print("="*70)
print(f"\nTime taken: {elapsed_time:.2f} seconds")
print(f"Total products: {len(products_df):,}")
print(f"File size: {products_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print("\n📊 DATA SUMMARY:")
print("-"*70)
print(f"Price range: ₹{products_df['Price'].min():,} - ₹{products_df['Price'].max():,}")
print(f"Average price: ₹{products_df['Price'].mean():,.0f}")
print(f"Average rating: {products_df['Rating'].mean():.2f}")
print(f"Products out of stock: {(products_df['Stock_Quantity'] == 0).sum()}")
print(f"Featured products: {products_df['Is_Featured'].sum()}")

print("\n📦 CATEGORY DISTRIBUTION:")
print(products_df['Category'].value_counts())

print("\n⭐ RATING DISTRIBUTION:")
rating_bins = pd.cut(products_df['Rating'], bins=[0, 2, 3, 4, 5], labels=['1-2', '2-3', '3-4', '4-5'])
print(rating_bins.value_counts().sort_index())

print("\n💰 DISCOUNT DISTRIBUTION:")
print(products_df['Discount_Percent'].value_counts().head(10))

print("\n⚠️ MISSING VALUES:")
print(products_df.isnull().sum()[products_df.isnull().sum() > 0])

print("\n📋 SAMPLE DATA (first 5 rows):")
print(products_df[['Product_ID', 'Product_Name', 'Category', 'Price', 'Rating', 'Stock_Quantity']].head())



GENERATING PRODUCTS.CSV

Generating 1,000 products...
  → Setting up product categories...
  → Generating product attributes...
  → Assembling dataframe...
  → Adding realistic missing values...
  → Saving to products.csv...

✅ PRODUCTS.CSV GENERATED SUCCESSFULLY!

Time taken: 0.18 seconds
Total products: 1,000
File size: 0.55 MB

📊 DATA SUMMARY:
----------------------------------------------------------------------
Price range: ₹393 - ₹150,000
Average price: ₹22,717
Average rating: 4.20
Products out of stock: 55
Featured products: 70

📦 CATEGORY DISTRIBUTION:
Category
Electronics               150
Clothing                  120
Grocery & Gourmet         120
Home & Kitchen            100
Health & Wellness         100
Beauty & Personal Care    100
Sports & Fitness           80
Books                      80
Automotive                 80
Toys & Games               70
Name: count, dtype: int64

⭐ RATING DISTRIBUTION:
Rating
1-2      0
2-3     23
3-4    309
4-5    668
Name: count, dtype: int